# Code

  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-cw5b76vf
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-cw5b76vf
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17532 sha256=6fa16f7d0130dd8f88c08e48e1b22818f91d13660954a456852e9c5f3989df6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-krmgq4v0/wheels/4b/f9/e9/82474a5d9a64702ba7dd5481dee05d267c69bcfa480d382b7c
Successfully built tvdatafeed


In [2]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from datetime import datetime
import time
import pygame

pygame.init()

sound_file = "Military_New_Message.mp3"  # Replace with the path to your sound file
pygame.mixer.music.load(sound_file)

In [3]:
username = 'mashgh2'
password = 'Amin1380ali'
# username, password
tv = TvDatafeed(username ,password )

In [4]:
def analize_data(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')

    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy'
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] )
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1)

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1)

    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1)

    def determine_finall_desions_4hour(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True"  and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True"  and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions_4h'] = df_XUD.apply(determine_finall_desions_4hour, axis=1)

    # print(df_XUD)
    #last candle must delete?

    # Save the DataFrame to a CSV file
    # df_XUD.to_csv('output.csv', index=True)
    # df_XUD.drop(df_XUD.index[-1], inplace=False)
    df_XUD = df_XUD.drop(df_XUD.index[-1])

    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [5]:
def analize_data_with_last_candle(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')

    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy'
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] )
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1)

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1)

    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1)

    def determine_finall_desions_4hour(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True"  and row["rule_4_5"] == "buy"):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True"  and row["rule_4_5"] == "sell"):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_4_5"] == "buy"):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions_4h'] = df_XUD.apply(determine_finall_desions_4hour, axis=1)

    # print(df_XUD)
    #last candle must delete?

    # Save the DataFrame to a CSV file
    # df_XUD.to_csv('output.csv', index=True)
    # df_XUD.drop(df_XUD.index[-1], inplace=False)

    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [6]:
def get_data_5_min(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_5_minute,n_bars=200)
    return df_XUD

def get_data_30_min(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_30_minute,n_bars=200)
    return df_XUD
def get_data_1_hour(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_1_hour,n_bars=200)
    return df_XUD
def get_data_4_hour(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_4_hour,n_bars=200)
    return df_XUD

def get_data_dayly(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_daily,n_bars=80)
    return df_XUD


def get_data_5_min_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_5_minute,n_bars=200)
    return df_XUD

def get_data_30_min_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_30_minute,n_bars=200)
    return df_XUD
def get_data_1_hour_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_1_hour,n_bars=200)
    return df_XUD
def get_data_4_hour_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_4_hour,n_bars=200)
    return df_XUD

def get_data_dayly_forex(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='OANDA',interval=Interval.in_daily,n_bars=80)
    return df_XUD

In [7]:

list_crypto_spot = ['GBPUSDT', 'ZECUSDT', 'XTZUSDT', 'FLOKIUSDT', 'KNCUSDT', 'NEOUSDT', 'THETAUSDT', 'ANKRUSDT', 'LAZIOUSDT',
                    'USTCUSDT', 'LRCUSDT', 'AMBUSDT', 'KSMUSDT', 'KLAYUSDT', 'DUSKUSDT', 'BTCUSDT', 'ETHUSDT', 'USDCUSDT', 'XRPUSDT',
                    'TUSDUSDT', 'BNBUSDT', 'BLZUSDT', 'CYBERUSDT', 'BUSDUSDT', 'SOLUSDT', 'RUNEUSDT', 'OPUSDT', 'LTCUSDT', 'MATICUSDT',
                    'DOGEUSDT', 'AKROUSDT', 'SHIBUSDT', 'ARBUSDT', 'SUIUSDT', 'EURUSDT', 'HBARUSDT', 'SEIUSDT', 'LEVERUSDT', 'UNFIUSDT',
                    'LINKUSDT', 'TRXUSDT', 'OXTUSDT', 'ADAUSDT', 'FILUSDT', 'SFPUSDT', 'BCHUSDT', 'AVAXUSDT', 'MKRUSDT', 'LINAUSDT',
                    'FTMUSDT', 'APEUSDT', 'WAVESUSDT', 'PEPEUSDT', 'ALPINEUSDT', 'CRVUSDT', 'YGGUSDT', 'FDUSDUSDT', 'WLDUSDT', 'ETCUSDT',
                    'BAKEUSDT', 'BNTUSDT', 'COMPUSDT', 'APTUSDT', 'FETUSDT', 'TOMOUSDT', 'XMRUSDT', 'DOTUSDT', 'LPTUSDT', 'STMXUSDT',
                    'INJUSDT', 'GALAUSDT', 'ATOMUSDT', 'UNIUSDT', 'RDNTUSDT', 'ASTRUSDT', 'POWRUSDT', 'LDOUSDT', 'MCUSDT', 'OGUSDT',
                    'CFXUSDT', 'XLMUSDT', 'MASKUSDT', 'RNDRUSDT', 'SANDUSDT', 'NEARUSDT', 'ARKMUSDT', 'EOSUSDT', 'RIFUSDT', 'DYDXUSDT',
                    'ACHUSDT', 'MANAUSDT', 'SNXUSDT', 'CAKEUSDT', 'VETUSDT', 'KAVAUSDT', 'SANTOSUSDT', 'AGIXUSDT', 'DREPUSDT',
                    'ICPUSDT', 'BELUSDT', 'MBOXUSDT', 'STXUSDT', 'AAVEUSDT', 'ALGOUSDT', 'LUNCUSDT', 'ACMUSDT', 'ARPAUSDT',
                    'GRTUSDT', 'IDUSDT', 'WRXUSDT', 'SUSHIUSDT', 'OGNUSDT', 'MTLUSDT', 'WOOUSDT', 'ARUSDT', 'ONEUSDT', 'MAGICUSDT',
                    'AVAUSDT', 'ANTUSDT', 'EDUUSDT', 'GMTUSDT', 'GALUSDT', 'TRUUSDT', 'FLOWUSDT', 'ROSEUSDT', 'JASMYUSDT', 'IMXUSDT',
                    'SXPUSDT', 'FXSUSDT', 'PORTOUSDT', 'EGLDUSDT', 'GMXUSDT', 'QNTUSDT', 'BETAUSDT', 'STGUSDT', 'MINAUSDT', 'C98USDT',
                    'LUNAUSDT', 'POLYXUSDT', 'HFTUSDT', 'XVGUSDT', 'PHAUSDT', 'MAVUSDT', 'AXSUSDT', 'HIGHUSDT', 'CHZUSDT', 'HOOKUSDT',
                    'SSVUSDT', 'DODOUSDT', 'ZILUSDT', 'ZECUSDT', 'GBPUSDT', 'XTZUSDT', 'FLOKIUSDT', 'KNCUSDT', 'THETAUSDT', 'NEOUSDT',
                    'AMBUSDT', 'LAZIOUSDT', 'USTCUSDT', 'LRCUSDT', 'KSMUSDT', 'ANKRUSDT', 'KLAYUSDT', 'DUSKUSDT', '1INCHUSDT', 'LITUSDT',
                    'IOTXUSDT', 'DARUSDT', 'OCEANUSDT', 'HIFIUSDT', 'BARUSDT', 'BSWUSDT', 'PAXGUSDT', 'RADUSDT', 'YFIUSDT', 'API3USDT',
                    'CHRUSDT', 'VOXELUSDT', 'ALICEUSDT', 'ENJUSDT', 'LQTYUSDT', 'FIDAUSDT', 'TWTUSDT', 'RSRUSDT', 'ELFUSDT', 'BATUSDT',
                    'AUCTIONUSDT', 'IOTAUSDT', 'PHBUSDT', 'XECUSDT', 'GTCUSDT', 'GLMUSDT', 'PENDLEUSDT', 'SYNUSDT', 'DASHUSDT', 'OSMOUSDT',
                    'AGLDUSDT', 'PLAUSDT', 'NMRUSDT', 'STORJUSDT', 'NKNUSDT', 'WINGUSDT', 'ALPACAUSDT', 'BANDUSDT', 'POLSUSDT', 'REQUSDT',
                    'ONTUSDT', 'AMPUSDT', 'ATMUSDT', 'USDPUSDT', 'JOEUSDT', 'ENSUSDT', 'KDAUSDT', 'CELOUSDT', 'JUVUSDT', 'MOVRUSDT',
                    'BTTCUSDT', 'CTSIUSDT', 'CTKUSDT', 'QTUMUSDT', 'TLMUSDT', 'ASRUSDT', 'COMBOUSDT', 'KEYUSDT', 'LOKAUSDT', 'CKBUSDT',
                    'ICXUSDT', 'HOTUSDT', 'AUDIOUSDT', 'ATAUSDT', 'REEFUSDT', 'PONDUSDT', 'OMGUSDT', 'FLUXUSDT', 'TUSDT', 'PYRUSDT',
                    'UMAUSDT', 'COTIUSDT', 'PEOPLEUSDT', 'GLMRUSDT', 'SKLUSDT', 'DENTUSDT', 'VGXUSDT', 'RENUSDT', 'RVNUSDT',
                    'BETHUSDT', 'RLCUSDT', 'CELRUSDT', 'BURGERUSDT', 'STPTUSDT', 'PSGUSDT', 'IDEXUSDT', 'NEXOUSDT', 'SNTUSDT',
                    'SCRTUSDT', 'BNXUSDT', 'ZENUSDT', 'ALPHAUSDT', 'WAXPUSDT', 'MDTUSDT', 'CITYUSDT', 'FLMUSDT', 'PERPUSDT', 'TFUELUSDT',
                    'DGBUSDT', 'ZRXUSDT', 'TRBUSDT', 'OAXUSDT', 'ILVUSDT', 'CLVUSDT', 'CHESSUSDT', 'ACAUSDT', 'BADGERUSDT', 'XVSUSDT',
                    'DIAUSDT', 'FRONTUSDT', 'MULTIUSDT', 'SLPUSDT', 'TVKUSDT', 'CVXUSDT', 'IOSTUSDT', 'OMUSDT', 'BONDUSDT', 'FIROUSDT',
                    'SPELLUSDT', 'FIROUSDT', 'XNOUSDT', 'VIDTUSDT', 'SUPERUSDT', 'XEMUSDT', 'ONGUSDT', 'GNSUSDT', 'VIBUSDT', 'MBLUSDT',
                    'MOBUSDT', 'QUICKUSDT', 'PNTUSDT', 'RAYUSDT', 'FORTHUSDT', 'OOKIUSDT', 'SCUSDT', 'HIVEUSDT', 'MDXUSDT', 'ERNUSDT',
                    'FISUSDT', 'TKOUSDT', 'REIUSDT', 'ORNUSDT', 'BALUSDT', 'DATAUSDT', 'ASTUSDT', 'TROYUSDT', 'STEEMUSDT', 'QIUSDT',
                    'BICOUSDT', 'RPLUSDT', 'BTSUSDT', 'WINUSDT', 'UFTUSDT', 'UTKUSDT', 'CTXCUSDT', 'FUNUSDT', 'KP3RUSDT', 'PROMUSDT',
                    'EPXUSDT', 'PERLUSDT', 'ALCXUSDT', 'DFUSDT', 'WBTCUSDT', 'SYSUSDT', 'JSTUSDT', 'PUNDIXUSDT', 'WANUSDT', 'LSKUSDT',
                    'DEGOUSDT', 'IRISUSDT', 'KMDUSDT', 'LTOUSDT', 'STRAXUSDT', 'PROSUSDT', 'RAREUSDT', 'HARDUSDT', 'CVCUSDT', 'DCRUSDT',
                    'COSUSDT', 'VITEUSDT', 'WNXMUSDT', 'NULSUSDT', 'DEXEUSDT', 'DOCKUSDT','AERGOUSDT', 'CVPUSDT', 'FARMUSDT', 'GHSTUSDT',
                    'LOOMUSDT', 'VTHOUSDT', 'GASUSDT', 'ARDRUSDT', 'GNOUSDT', 'FORUSDT', 'QKCUSDT', 'SUNUSDT', 'FIOUSDT', 'MLNUSDT',
                    'WTCUSDT', 'BIFIUSDT', 'WBETHUSDT']


def extract_signal_5_min(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0



def extract_signal_30_min(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", " ]",end=" ")
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0


def extract_signal_1_hour(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0

def extract_signal_4_hour(df_XUD,time):
    if df_XUD["finall_desions_4h"][-1] == "buy" or df_XUD["finall_desions_4h"][-1] == "buy_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "buy", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    elif df_XUD["finall_desions_4h"][-1] == "sell" or df_XUD["finall_desions_4h"][-1] == "sell_half" :
        print("[" , df_XUD["symbol"][-1].split(':')[1] , "sell", " ]",end=" ")
        df_XUD.to_csv(f'output_{df_XUD["symbol"][-1]}.csv'  , index = True)
    else :
        return 0

In [8]:
import concurrent.futures
import multiprocessing

def analyze_crypto_5min(crypto):
    df_XUDD = analize_data(get_data_5_min(crypto))
    extract_signal_5_min(df_XUDD, datetime.now().minute)
def analyze_crypto_30min(crypto):
    df_XUDD = analize_data(get_data_30_min(crypto))
    extract_signal_30_min(df_XUDD, datetime.now().minute)
def analyze_crypto_1hour(crypto):
    df_XUDD = analize_data(get_data_1_hour(crypto))
    extract_signal_1_hour(df_XUDD, datetime.now().minute)
def analyze_crypto_4hour(crypto):
    df_XUDD = analize_data(get_data_4_hour(crypto))
    extract_signal_4_hour(df_XUDD, datetime.now().minute)

def analyze_pair_coin_5min(pair_coin):
    df_XUDD = analize_data(get_data_5_min_forex(pair_coin))
    extract_signal_5_min(df_XUDD, datetime.now().minute)

def analyze_pair_coin_30min(pair_coin):
    df_XUDD = analize_data(get_data_30_min_forex(pair_coin))
    extract_signal_30_min(df_XUDD, datetime.now().minute)

def analyze_pair_coin_1hour(pair_coin):
    df_XUDD = analize_data(get_data_1_hour_forex(pair_coin))
    extract_signal_1_hour(df_XUDD, datetime.now().minute)
def analyze_pair_coin_4hour(pair_coin):
    df_XUDD = analize_data(get_data_4_hour_forex(pair_coin))
    extract_signal_4_hour(df_XUDD, datetime.now().minute)


In [9]:

def extract_break_SMA20_4h(df_XUD,time):
    if df_XUD["inside candle (rule 1)"][-1] == "buy":
        # print("time :" , time ,df_XUD["symbol"][-1] , "buy" , "4 hour time frame , looking 5 min to enter")
        return 1
    elif df_XUD["inside candle (rule 1)"][-1] == "sell":
        # print("time :" , time ,df_XUD["symbol"][-1], "sell" , "4 hour time frame , looking 5 min to enter")
        return -1
    else :
        return 0

def extract_break_SMA20_1h_buy(df_XUD):
    if df_XUD["inside candle (rule 1)"][-1] == "buy"  :
        # print("time :" , time ,df_XUD["symbol"][-1] , "buy" , "4 hour time frame , looking 5 min to enter")
        return 1
    else :
        return 0

def extract_break_SMA20_1h_sell(df_XUD):
    if df_XUD["inside candle (rule 1)"][-1] == "sell":
        # print("time :" , time ,df_XUD["symbol"][-1], "sell" , "4 hour time frame , looking 5 min to enter")
        return 1
    else :
        return 0
def analyze_pair_coin_dayly_1hour(pair_coin):
    df_XUDD = analize_data(get_data_dayly_forex(pair_coin))
    return extract_break_SMA20_4h(df_XUDD, datetime.now().minute)

def find_accepted_entry_1_hour_crypto(coin,position_type):
    df_XUDD = analize_data(get_data_1_hour(coin))
    if position_type == "buy":
      return extract_break_SMA20_1h_buy(df_XUDD)
    elif position_type == "sell":
      return extract_break_SMA20_1h_sell(df_XUDD)
    else:
      return 0

In [10]:
# caculate change_percent

import pandas as pd
from datetime import datetime

# Get today's date

def caculate_change_percent(price_open_of_the_new_day,live_price):
  return abs(price_open_of_the_new_day - live_price) / price_open_of_the_new_day *100

def change_percent(df_XUD):
  today_date = datetime.now().date()
  # Extract the date part (midnight) from today's date
  price_open_of_the_new_day = df_XUD[df_XUD.index.date == today_date].iloc[0]["open"]
  return caculate_change_percent(price_open_of_the_new_day ,df_XUD.iloc[-1]["close"] )

def change_percent_daily(df_XUD):
  return caculate_change_percent(df_XUD["open"][-1] ,df_XUD["close"][-1] )


In [11]:

def analyze_crypto_dayly_1hour(crypto):
    df_XUDD = analize_data_with_last_candle(get_data_dayly(crypto))
    return extract_break_SMA20_4h(df_XUDD, datetime.now().minute) , change_percent_daily(df_XUDD)


In [ ]:
import asyncio
from telethon.sync import TelegramClient

import nest_asyncio
import asyncio
from telethon.sync import TelegramClient

nest_asyncio.apply()  # Allow running asyncio event loop in Jupyter

async def send_telegram_message(api_id, api_hash, phone_number, message):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        await client.send_message(phone_number, message)

api_id = 2420963
api_hash = 'f0a28529c1aff3bf8b30e53e45030813'
phone_number = 'me'

In [ ]:
    one_time =1
    one_time_v2 =0
    counter_24_1hour = 0
    commen_5_30 =[]
    commen_5_1hour =[]
    commen_1hour_30 =[]
    while(True):
      current_time = datetime.now()
      minutes_5 = int(str(current_time.minute))%5
      minutes_30 = int(str(current_time.minute))%30
      minutes_1_hour = int(str(current_time.minute))
      hour = current_time.hour
      hour_4  = int(str(current_time.hour))%4
      second  = current_time.second
      # 4 hour sma break
      ## set 5 min rule
      if ((minutes_5 == 0) and second <= 1 and counter_24_1hour > 1 ) or one_time_v2 == 1:
        one_time_v2 = 0
        counter_24_1hour = counter_24_1hour - 1
        del_buy_crypto = []
        del_sell_crypto = []
        print("dayly approved and 1 hour entry founded must act quakly")
        for i in range(len(golden_list_crypto_buy)):
          if find_accepted_entry_1_hour_crypto(golden_list_crypto_buy[i],"buy") == True:
              message = "time to buy" + golden_list_crypto_buy[i]
              await send_telegram_message(api_id, api_hash, phone_number, message)
              del_buy_crypto.append(golden_list_crypto_buy[i])
        # delete list

        for item in del_buy_crypto:
           if item in golden_list_crypto_buy:
              golden_list_crypto_buy.remove(item)

        for i in range(len(golden_list_crypto_sell)):
          if find_accepted_entry_1_hour_crypto(golden_list_crypto_sell[i],"sell") == True:
              message = "time to buy" + golden_list_crypto_buy[i]
              await send_telegram_message(api_id, api_hash, phone_number, message)
              del_sell_crypto.append(golden_list_crypto_sell[i])
        # delete list
        for item in del_sell_crypto:
           if item in golden_list_crypto_sell:
              golden_list_crypto_sell.remove(item)
        # play sound if it found anything
        if del_sell_crypto != [] or del_buy_crypto!= []:
           pygame.mixer.music.play()
        if counter_4hour == 0:
          golden_list_crypto_buy = []
          golden_list_crypto_sell = []




      ## set dayly sma rule
      if ((minutes_1_hour == 0 and second >=58 and hour== 20) or one_time == 1 ):
          one_time = 0
          one_time_v2 = 1
          counter_4hour = 28
          golden_list_crypto_sell = []
          golden_list_crypto_buy = []
          golden_list_sell = []
          golden_list_buy = []
          for i in range(len(list_crypto_spot)):
              position,change = analyze_crypto_dayly_1hour(list_crypto_spot[i])
              if position == 1:
                golden_list_crypto_buy.append(list_crypto_spot[i])
                golden_list_buy.append([list_crypto_spot[i] , "ch per = ","{:.2f}".format((change))])

              elif position == -1:
                golden_list_crypto_sell.append(list_crypto_spot[i])
                golden_list_sell.append([list_crypto_spot[i] , "ch per = ","{:.2f}".format((change))])
          print("golden list \n  buy list :",golden_list_buy,'\n sell list :' ,golden_list_sell)

In [12]:
from datetime import datetime


def main():
    one_time =1
    one_time_v2 =0
    counter_24_1hour = 0
    commen_5_30 =[]
    commen_5_1hour =[]
    commen_1hour_30 =[]
    while(True):
      current_time = datetime.now()
      minutes_5 = int(str(current_time.minute))%5
      minutes_30 = int(str(current_time.minute))%30
      minutes_1_hour = int(str(current_time.minute))
      hour = current_time.hour
      hour_4  = int(str(current_time.hour))%4
      second  = current_time.second
      # 4 hour sma break
      ## set 5 min rule
      if ((minutes_5 == 0) and second <= 1 and counter_24_1hour > 1 ) or one_time_v2 == 1:
        one_time_v2 = 0
        counter_24_1hour = counter_24_1hour - 1
        del_buy_crypto = []
        del_sell_crypto = []
        print("dayly approved and 1 hour entry founded must act quakly")
        for i in range(len(golden_list_crypto_buy)):
          if find_accepted_entry_1_hour_crypto(golden_list_crypto_buy[i],"buy") == True:
              del_buy_crypto.append(golden_list_crypto_buy[i])
        # delete list

        for item in del_buy_crypto:
           if item in golden_list_crypto_buy:
              golden_list_crypto_buy.remove(item)

        for i in range(len(golden_list_crypto_sell)):
          if find_accepted_entry_1_hour_crypto(golden_list_crypto_sell[i],"sell") == True:
              del_sell_crypto.append(golden_list_crypto_sell[i])
        # delete list
        for item in del_sell_crypto:
           if item in golden_list_crypto_sell:
              golden_list_crypto_sell.remove(item)
        # play sound if it found anything
        if del_sell_crypto != [] or del_buy_crypto!= []:
           pygame.mixer.music.play()
        if counter_4hour == 0:
          golden_list_crypto_buy = []
          golden_list_crypto_sell = []




      ## set dayly sma rule
      if ((minutes_1_hour == 0 and second >=58 and hour== 20) or one_time == 1 ):
          one_time = 0
          one_time_v2 = 1
          counter_4hour = 28
          golden_list_crypto_sell = []
          golden_list_crypto_buy = []
          golden_list_sell = []
          golden_list_buy = []
          for i in range(len(list_crypto_spot)):
              position,change = analyze_crypto_dayly_1hour(list_crypto_spot[i])
              if position == 1:
                golden_list_crypto_buy.append(list_crypto_spot[i])
                golden_list_buy.append([list_crypto_spot[i] , "ch per = ","{:.2f}".format((change))])

              elif position == -1:
                golden_list_crypto_sell.append(list_crypto_spot[i])
                golden_list_sell.append([list_crypto_spot[i] , "ch per = ","{:.2f}".format((change))])
          print("golden list \n  buy list :",golden_list_buy,'\n sell list :' ,golden_list_sell)

if __name__ == "__main__" :
    main()

golden list 
  buy list : [['ZECUSDT', 'ch per = ', '5.17'], ['XTZUSDT', 'ch per = ', '4.53'], ['BTCUSDT', 'ch per = ', '6.61'], ['ETHUSDT', 'ch per = ', '4.79'], ['TUSDUSDT', 'ch per = ', '0.02'], ['BNBUSDT', 'ch per = ', '4.48'], ['RUNEUSDT', 'ch per = ', '9.14'], ['OPUSDT', 'ch per = ', '5.98'], ['EURUSDT', 'ch per = ', '0.16'], ['TRXUSDT', 'ch per = ', '1.26'], ['ADAUSDT', 'ch per = ', '3.40'], ['BCHUSDT', 'ch per = ', '16.21'], ['FTMUSDT', 'ch per = ', '6.95'], ['ETCUSDT', 'ch per = ', '6.47'], ['FETUSDT', 'ch per = ', '3.04'], ['DOTUSDT', 'ch per = ', '1.89'], ['ASTRUSDT', 'ch per = ', '0.99'], ['NEARUSDT', 'ch per = ', '3.43'], ['RIFUSDT', 'ch per = ', '4.15'], ['ICPUSDT', 'ch per = ', '5.78'], ['AAVEUSDT', 'ch per = ', '2.95'], ['ONEUSDT', 'ch per = ', '4.17'], ['ROSEUSDT', 'ch per = ', '5.42'], ['QNTUSDT', 'ch per = ', '0.59'], ['XVGUSDT', 'ch per = ', '7.74'], ['ZECUSDT', 'ch per = ', '5.09'], ['XTZUSDT', 'ch per = ', '4.53'], ['OCEANUSDT', 'ch per = ', '3.98'], ['ALICEUSDT',

KeyboardInterrupt: ignored